<a href="https://colab.research.google.com/github/ParasKanchan/Sentiment_Analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import re
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

import torch
from torch.utils.data import Dataset

from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments
)


In [5]:
df = pd.read_csv('sentiment_reviews.csv')
df.head()


,review,label
0,The product is excellent and works perfectly,1
1,"Terrible experience, will not buy again",0
2,Exceeded my expectations,1
3,Extremely poor performance,0
4,Very happy with the purchase,1


In [7]:
def basic_preprocess(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_review"] = df["review"].apply(basic_preprocess)
df.head()


,review,label,clean_review
0,The product is excellent and works perfectly,1,the product is excellent and works perfectly
1,"Terrible experience, will not buy again",0,terrible experience will not buy again
2,Exceeded my expectations,1,exceeded my expectations
3,Extremely poor performance,0,extremely poor performance
4,Very happy with the purchase,1,very happy with the purchase


In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_review"],
    df["label"],
    test_size=0.25,
    random_state=42,
    stratify=df["label"]
)


In [9]:
ml_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1, 2),
        stop_words="english"
    )),
    ("clf", LogisticRegression(max_iter=1000))
])

ml_pipeline.fit(X_train, y_train)

y_pred_ml = ml_pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_ml))
print(classification_report(y_test, y_pred_ml, digits=3))


Accuracy: 1.0
              precision    recall  f1-score   support

           0      1.000     1.000     1.000       125
           1      1.000     1.000     1.000       125

    accuracy                          1.000       250
   macro avg      1.000     1.000     1.000       250
weighted avg      1.000     1.000     1.000       250



In [11]:
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    df["review"],
    df["label"],
    test_size=0.25,
    random_state=42,
    stratify=df["label"]
)


In [12]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = list(texts)
        self.labels = list(labels)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long)
        }

train_dataset = SentimentDataset(X_train_raw, y_train_raw, tokenizer)
test_dataset = SentimentDataset(X_test_raw, y_test_raw, tokenizer)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, preds)}


In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [18]:
training_args = TrainingArguments(
    output_dir="./bert_output",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,


    eval_strategy="epoch",
    logging_steps=20,
    learning_rate=2e-5,
    weight_decay=0.02

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.000300,0.000156,1.000000
2,0.000200,0.000096,1.000000


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=376, training_loss=0.001822322639240388, metrics={'train_runtime': 1318.2202, 'train_samples_per_second': 1.138, 'train_steps_per_second': 0.285, 'total_flos': 49333322880000.0, 'train_loss': 0.001822322639240388, 'epoch': 2.0})

In [19]:
results = trainer.evaluate()
results


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 9.637223411118612e-05,
 'eval_accuracy': 1.0,
 'eval_runtime': 62.3365,
 'eval_samples_per_second': 4.01,
 'eval_steps_per_second': 1.011,
 'epoch': 2.0}

In [20]:
def predict_with_bert(texts):
    model.eval()
    enc = tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=64,
        return_tensors="pt"
    )
    with torch.no_grad():
        out = model(
            input_ids=enc["input_ids"],
            attention_mask=enc["attention_mask"]
        )
        probs = torch.softmax(out.logits, dim=-1)
        preds = torch.argmax(probs, dim=-1).numpy()
    return preds, probs.numpy()

example = ["I loved the movie!", "Worst film ever."]
preds, probs = predict_with_bert(example)

list(zip(example, preds, probs))


[('I loved the movie!',
  np.int64(1),
  array([0.00101786, 0.99898213], dtype=float32)),
 ('Worst film ever.',
  np.int64(0),
  array([9.9987578e-01, 1.2414281e-04], dtype=float32))]